In [1]:
import pygrib as pg
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import os
import datetime
import pyproj
from scipy import stats
import gzip

import dataclasses as dc

In [3]:
torprob_file = pathlib.Path('./pp_forecasts/regProbs/19500428_1630.npz')
ndfd_area = 25
nsims = 10000
tornado_direction_distribution = stats.norm(50,15)
coolseason = [1,2,3,4,11,12]

In [4]:
os.environ['IMPACTSDATA'] = "../impacts-data"
impacts_data_root = pathlib.Path(os.environ['IMPACTSDATA'],"pas-input-data").expanduser().resolve()

In [5]:
def read_pp_file(file):
    """ Read a PP file and format properly for the PAS """
    data = np.load(file)
    probs = data['fcst']*100
    vals = np.where(probs==0, -1.,probs)
    
    return vals

In [6]:
outdir = pathlib.Path("..", "output").resolve()
outdir.mkdir(exist_ok=True)

In [7]:
### Setup Simulation ###
# Determine Date/Time of outlook from filename
date_in_name = torprob_file.name.split("/")[-1].split(".")[0]
dt = datetime.datetime.strptime(date_in_name, "%Y%m%d_%H%M")
outfile = outdir.joinpath(f"{dt.strftime('%Y%m%d%H%M')}_pp.psv.gz")

In [77]:
torn = read_pp_file(torprob_file)

In [78]:
np.unique(torn)

array([-1.,  2.,  5., 10.])

In [79]:
try:
    continuous_torn = dc.make_continuous(torn)
except ValueError:
    import sys
    if torn.max() == 0:
        with gzip.GzipFile(outfile, "w") as OUT:
            OUT.write("".encode())
        sys.exit(0)
    else:
        print("There was an uncaught error converting to continuous probabilities. Exiting...")
        sys.exit(1)

In [82]:
sigtorn = read_pp_file(pathlib.Path('./pp_forecasts/sigProbs/19500428_1630.npz')).astype(int)
np.unique(sigtorn)

array([-1, 10])

In [83]:
sigtorn[sigtorn > 0] = 1
np.unique(sigtorn)

array([-1,  1])

In [61]:
np.max(sigtorn)

1

In [62]:
np.unique(torn)

array([-1,  2,  5, 10])

Make double sig

In [84]:
if (torn.max() >= 30) and (sigtorn.max() > 0):
    sigtorn[torn >= 15] += 1

In [86]:
sigtorn_1d = sigtorn.ravel()
usesig = True if (dt.month in coolseason) or (sigtorn.max() > 0) else False

In [87]:
np.unique(sigtorn_1d)

array([-1,  1])

In [88]:
### Run Tornado Count Simulation ###
print(f"Running {nsims:,d} Tornado Count Simulations")
tornado_dists = dc.TornadoDistributions()
counts = np.zeros((5, nsims), dtype=int)
counts[0, :] = (tornado_dists.f02.rvs(nsims) * ndfd_area * (torn == 2).sum()).astype(int)
counts[1, :] = (tornado_dists.f05.rvs(nsims) * ndfd_area * (torn == 5).sum()).astype(int)
counts[2, :] = (tornado_dists.f10.rvs(nsims) * ndfd_area * (torn == 10).sum()).astype(int)
counts[3, :] = (tornado_dists.f15.rvs(nsims) * ndfd_area * (torn == 15).sum()).astype(int)
counts[4, :] = (tornado_dists.f30.rvs(nsims) * ndfd_area * (torn >= 30).sum()).astype(int)

Running 10,000 Tornado Count Simulations


*counts* is a 5x10000 array to hold the number of tors for each sim for each prob level

In [66]:
dc.weighted_choice(2,torn,continuous_torn,10)

array([234419, 227955, 330934, 338544, 381482, 341700, 404000, 292294,
       319225, 207572])

In [89]:
### Setup Impact Simulation ###
igrids = dc.ImpactGrids(impacts_data_root)

scounts = counts.sum(axis=1)
inds02 = dc.weighted_choice(prob=2, probs=torn, cprobs=continuous_torn, size=scounts[0])
inds05 = dc.weighted_choice(prob=5, probs=torn, cprobs=continuous_torn, size=scounts[1])
inds10 = dc.weighted_choice(prob=10, probs=torn, cprobs=continuous_torn, size=scounts[2])
inds15 = dc.weighted_choice(prob=15, probs=torn, cprobs=continuous_torn, size=scounts[3])
inds30 = dc.weighted_choice(prob=30, probs=torn, cprobs=continuous_torn, size=scounts[4])
inds = dc.flatten_list([inds02, inds05, inds10, inds15, inds30])

In [111]:
sigtorn_1d[365315] == -1
#np.where(sigtorn_1d == 1)

True

In [117]:
non_sig_inds = sigtorn_1d[inds] == -1
single_sig_inds = sigtorn_1d[inds] == 1
double_sig_inds = sigtorn_1d[inds] == 2

if usesig:
    single_sig_inds += non_sig_inds
    non_sig_inds[:] = False

In [114]:
np.unique(non_sig_inds)

array([False])

In [118]:
# Handle Locations
non_sig_loc_inds = inds[non_sig_inds]
single_sig_loc_inds = inds[single_sig_inds]
double_sig_loc_inds = inds[double_sig_inds]

In [103]:
non_sig_loc_inds

array([], dtype=int64)

In [119]:
# Handle Ratings
_mags=[0, 1, 2, 3, 4, 5]
non_sig_ratings = np.random.choice(_mags, size=non_sig_inds.sum(),
                                   replace=True, p=tornado_dists.r_nonsig)
single_sig_ratings = np.random.choice(_mags, size=single_sig_inds.sum(),
                                      replace=True, p=tornado_dists.r_singlesig)
double_sig_ratings = np.random.choice(_mags, size=double_sig_inds.sum(),
                                      replace=True, p=tornado_dists.r_doublesig)

In [34]:
non_sig_ratings

array([], dtype=int64)

In [120]:
# Handle Distances
non_sig_distances = dc.get_distances(non_sig_ratings, tornado_dists)
single_sig_distances = dc.get_distances(single_sig_ratings, tornado_dists)
double_sig_distances = dc.get_distances(double_sig_ratings, tornado_dists)

In [33]:
non_sig_distances

array([], dtype=float64)

In [121]:
print("Running simulations...")
print("    Non Sig...")
non_sig = dc.simulate(non_sig_loc_inds, non_sig_distances,
                      non_sig_ratings, tornado_direction_distribution, igrids)
print("    Single Sig...")
single_sig = dc.simulate(single_sig_loc_inds, single_sig_distances,
                         single_sig_ratings, tornado_direction_distribution, igrids)
print("    Double Sig...")
double_sig = dc.simulate(double_sig_loc_inds, double_sig_distances,
                         double_sig_ratings, tornado_direction_distribution, igrids)

print("Splitting simulations back out...")
simulated_tornadoes = dc.flatten_list([non_sig, single_sig, double_sig])
np.random.shuffle(simulated_tornadoes)
_sims = np.split(simulated_tornadoes, counts.sum(axis=0).cumsum())[:-1]
realizations = dc.Realizations([dc.SyntheticTornadoRealization(_sim, i+1) for i, _sim in enumerate(_sims)])


print("Writing Out gzipped PSV file...")
with gzip.GzipFile(outfile, "w") as OUT:
    OUT.write(realizations.as_psv.encode())

Running simulations...
    Non Sig...
    Single Sig...
    Double Sig...
Splitting simulations back out...
Writing Out gzipped PSV file...


In [28]:
len(simulated_tornadoes)

7996

In [52]:
igrids = dc.ImpactGrids(impacts_data_root)

In [56]:
np.unique(igrids.wfo)

array(['0', 'ABQ', 'ABR', 'AKQ', 'ALY', 'AMA', 'APX', 'ARX', 'BGM', 'BIS',
       'BMX', 'BOI', 'BOU', 'BOX', 'BRO', 'BTV', 'BUF', 'CAE', 'CAR',
       'CHS', 'CLE', 'CRP', 'CTP', 'CYS', 'DDC', 'DLH', 'DMX', 'DTX',
       'DVN', 'EAX', 'EKA', 'EPZ', 'EWX', 'FFC', 'FGF', 'FGZ', 'FSD',
       'FWD', 'GGW', 'GID', 'GJT', 'GLD', 'GRB', 'GRR', 'GSP', 'GYX',
       'HGX', 'HNX', 'HUN', 'ICT', 'ILM', 'ILN', 'ILX', 'IND', 'IWX',
       'JAN', 'JAX', 'JKL', 'KEY', 'LBF', 'LCH', 'LIX', 'LKN', 'LMK',
       'LOT', 'LOX', 'LSX', 'LUB', 'LWX', 'LZK', 'MAF', 'MEG', 'MFL',
       'MFR', 'MHX', 'MKX', 'MLB', 'MOB', 'MPX', 'MQT', 'MRX', 'MSO',
       'MTR', 'OAX', 'OHX', 'OKX', 'OTX', 'OUN', 'PAH', 'PBZ', 'PDT',
       'PHI', 'PQR', 'PSR', 'PUB', 'RAH', 'REV', 'RLX', 'RNK', 'SEW',
       'SGF', 'SGX', 'SHV', 'SJT', 'SJU', 'TAE', 'TBW'], dtype='<U21')

In [24]:
newCWA = pathlib.Path('../impacts-data/pas-input-data/cwas.npz')
wfo = np.load(newCWA)

In [25]:
for key in wfo.keys():
    print(key)

cwas


In [20]:
outfile

PosixPath('/Users/josephpicca/Desktop/work2020/cimms-spc/IMPACTS-work/impacts/output/195004281630_pp.psv.gz')